---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
 1. 04/20/2009; 04/20/09; 4/20/09; 4/3/09
 2. Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
 3. 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
 4. Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
 5. Feb 2009; Sep 2009; Oct 2010
 6. 6/2008; 12/2009
 7. 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head()

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
dtype: object

In [2]:
def date_sorter():
    import numpy as np
    import pandas as pd
    import re
    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)
    df = pd.Series(doc)
    df = pd.DataFrame(df)
    df.columns = ['Note']
    df['Date'] = df['Note'].str.extract('(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})')
    df['Text second'] = df['Note'].str.extract('(\d{0,2}[.,]* (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.,]* \d{4})')
    df['Text first'] = df['Note'].str.extract('((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.,]* \d{1,2}[th]*[.,]* \d{2,4})')
    df['Month'] = df['Note'].str.extract('(\d{1,2}[/-]\d{2,4})')
    df['Month text'] = df['Note'].str.extract('((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[.,]* \d{4})')
    df['Year'] = df['Note'].str.extract('(\d{4})')
    for column in ['Text first', 'Text second', 'Month', 'Month text', 'Year']:
        df['Date'].fillna(df[column], inplace=True)
    #df.loc[248, 'Date'] = 'July 1995'
    #df.loc[80, 'Date'] = 'October 1, 2015'
    #df.loc[99, 'Date'] = 'November 1, 2014'
    #df.loc[271, 'Date'] = 'August 1, 2008'
    #df.loc[272, 'Date'] = 'February 1, 1993'
    df.loc[298, 'Date'] = 'January 1, 1993'
    df.loc[313, 'Date'] = 'December 1, 1978'
    #df.loc[321, 'Date'] = 'June 2, 1999'
    df['Datetime'] = pd.to_datetime(df['Date'], errors = 'coerce', infer_datetime_format= True)

    df.reset_index(inplace = True)
    df.sort_values(by = ['Datetime', 'index'], inplace = True)
    df.reset_index(drop= True, inplace = True)
    return df['index']
date_sorter()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if sys.path[0] == '':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  del sys.path[0]
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this w

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Name: index, Length: 500, dtype: int64